# 재귀와 패턴 매칭

재귀와 패턴 매칭을 간단한 예제를 통해 설명한다. 
보다 복잡한 예제와 설명은 강의를 진행하면서 차차 이루어질 것이다.

## 재귀

재귀(recursion)의 사전적 정의는 되풀이, 순환반복, 재발 등을 의미한다. 
그리고 재귀 함수는 재귀로 정의된 함수를 가리킨다.

그런데 재귀로 정의된 함수란 무엇일까?
다음 예제를 살펴보자. 

In [1]:
nonStoppingFtn x = nonStoppingFtn (x + 1)

`nonStoppingFtn` 함수의 본체에 `nonStoppingFtn` 이 사용된다.
이렇듯 함수를 정의할 때 함수의 본체에 자신이 사용되는 것을 재귀라 부른다.
즉, `nonStoppingFtn` 함수처럼 재귀로 정의된 함수를 재귀함수라 부른다. 

사실 `nonStoppingFtn` 함수는 인위적으로 구현된 함수이다. 
Coq, Agda와 같은 증명보조기(proof assistant)가 사용하는 프로그래밍 언어에서는
`nonStoppingFtn` 같은 함수는 애초부터 정의를 받아들이지 않는다.

**주의:** "정의를 받아들이지 않는다" 는 표현을 여기서는 일종의 문법 오류 정도로 이해하면 된다.

반면에 이전에 살펴 본 `sum_`, `qsort_`, `actionSeqn` 함수 또한 모두 재귀 함수들이다. 
하지만 이들은 `nonStoppingFtn`과 다른 종류의 재귀를 사용하는데,
바로 패턴 매칭과 함께 재귀가 사용된다. 

하스켈과 같은 함수형 프로그래밍 언어에서 재귀와 패턴 매칭은 매우 중요한 요소이며 서로 긴밀히 공존하는 관계이다.
여기서는 패턴 매칭과 재귀의 긴밀한 관계를 알아보기 위해 `sum_` 함수의 정의를 분석한다.

## 패턴 매칭과 재귀

`sum_` 함수의 정의는 다음과 같다.

In [2]:
sum_ :: [Integer] -> Integer
sum_ []= 0
sum_ (n:ns) = n + sum_ ns

Line 2: Use foldr
Found:
sum_ [] = 0
sum_ (n : ns) = n + sum_ ns
Why not:
sum_ ns = foldr (+) 0 ns

위 정의는 다음을 말하고 있다. 

* 공리스트인 경우: 모든 항목들의 합은 0이다.
* 리스트의 첫째 항목이 `n`이고 나머지 항목들로 구성된 리스트가 `ns`인 경우: 
    모든 항목들의 합은 `n`과 나머지 항목들의 합을 더한 값이다. 
    
좀 더 수학적으로 표현하면 다음과 같다.

$$
\text{sum}_{-}(\text{xs})= 
\begin{cases}
0,  & \text{if}\;\; \text{xs = []}\\
\text{n} + \text{sum}_{-}(\text{ns}), &  \text{if}\;\; \text{xs = n:ns}
\end{cases}
$$

즉, `sum_` 인자의 형태를 이용하면서 동시에 재귀를 이용하여 정의되었다. 
그런데 하스켈 인터프리터는 리스트의 형태(패턴)를 어떻게 파악할까? 

파이썬, 자바 등의 언어는 위와 같은 형식의 정의를 지원하지 않는다. 
파이썬의 경우 `sum_` 함수는 보통 다음과 같이 정의한다. 

```python
def sum_(xs):
    if len(xs) == 0:
        return 0
    else:
        return xs[0] + sum_(xs[1:])
```

즉, 인자의 형태를 보고 판단하는 것이 아니라 인자로 들어온 값의 길이에 따라
처리 방식을 달리한다.
예를 들어, 길이가 0이 아니면, 인자가 하나 이상일 것이기 때문에 첫째 항목과 나머지로
구분할 수 있다고 가정하고 그에 맞춘 계산을 지정한다. 

다시 말해, 파이썬, 자바 등의 명령형 프로그래밍 언어는 인자가 어떤 형태인지 파악하지 않거나 못한다. 
아마도 후자일 것이며, 이유는 자료형을 구현할 때,
정의된 자료형을 구조적으로 파악할 수 있는 도구를 함께 제공하지 못하는 방식을 사용하기 때문일 것이다. 

반면에 하스켈 등의 함수형 프로그래밍 언어는 애초부터 구조적으로 파악할 수 있는 방식으로
자료형을 정의할 수 있도록 하며,
**재귀적 유형**(recursive types)이 여기에 포함된다.

재귀적 유형에 대한 자세한 설명은 이후에 다룰 것이다.
여기서는 정수들로 구성된 리스트들의 유형을 재귀적 유형으로 정의하는 방법과 함께
`sum_` 함수를 패턴 매칭을 이용하여 재귀 함수로 정의하는 과정을 밑바닥부터 자세히 알아본다. 

### 재귀적 유형: 정수들의 리스트 구현하기

정수들로 구성되는 리스트들의 유형을 재귀적 유형으로 구현하는 방식은 다음과 같다. 

In [3]:
data IntList =   Nil 
               | Cons Integer IntList

위 정의에서 `Nil`과 `Cons`는 구성자(constructor)라고 불린다.
구성자들 사이의 구분은 **파이프**라 불리는 
작대기 모양의 기호(`'|'`)에 의해 이루어진다. 
엄밀히 말해 구성자는 모두 함수이며,
해당 유형의 값을 생성하는 도구 역할을 수행한다.

`IntList`에 사용된 두 개의 구성자의 역할은 다음과 같다.

* `Nil`은 인자를 받지 않는 함수, 즉, 하나의 상수(constant)이며, 
    여기서는 공리스트를 가리킨다.
* `Cons`는 인자 두 개를 받는 함수이며, 여기서는 하나의 정수와 다른 정수들의 리스트를 받아
    새로운 리스트를 만드는 함수를 가리킨다. 

**주의:** `Cons`의 둘째 인자가 `IntList` 유형의 값이어야 한다. 
즉, `IntList`는 재귀를 이용하며, 따라서 재귀적 유형에 속한다. 

결론적으로, `IntList` 유형을 갖는 값은 아래 두 가지 방식 중 하나를 이용해서만 생성할 수 있으며,
다른 방식은 허용되지 않는다.
* 첫째 방식: `Nil` 자체가 `IntList` 유형을 갖는 값이다
* 둘째 방식: `n`이 `Integer` 유형의 값, 즉, 하나의 정수이고,
    `ns`가 (이미) `IntList`의 유형의 값이면 
    `Cons n ns` 또한 `IntList` 유형의 값이다.
        
**참고:** Cons는 construct의 줄임말이며,
    프로그래밍언어론 분야에서 많이 사용되는 표현이다.
    `Cons n ns`가 `n`과 `ns`를 이용하여 새로운 리스트를 생성한다는 의미와 일맥상통한다.

#### 예제

`IntList` 유형을 갖는 값들은 다음과 같다.

In [4]:
-- []에 해당
:t Nil

Nil :: IntList

In [5]:
-- [1] 해당하는 값
:t (Cons 1 Nil)

(Cons 1 Nil) :: IntList

In [6]:
-- [1, 2]에 해당하는 값
:t (Cons 1 (Cons 2 Nil))

(Cons 1 (Cons 2 Nil)) :: IntList

In [7]:
-- [1, 2, 3]에 해당하는 값
:t (Cons 1 (Cons 2 (Cons 3 Nil)))

(Cons 1 (Cons 2 (Cons 3 Nil))) :: IntList

### 패턴 매칭: `sum_` 함수 다시 구현하기

앞서 구현한 정수들의 리스트에 포함된 항목들의 합을 계산하는 함수를 인자의 형태(패턴)만을 보고서
구현해보자. 
이처럼 인자의 형태에 따라 값을 지정하는 기법을 **패턴 매칭**(pattern matching)이라 부른다. 

In [8]:
sumInt :: IntList -> Integer
sumInt Nil = 0
sumInt (Cons n ns) = n + sumInt ns

위 정의는 `sum_`의 정의와 사실상 동일하다. 

* `Nil`: `[]`에 대응
* `Cons n ns`: `(n:ns)`에 대응

다만 `IntList`와 `[Integer]`가 다를 뿐이다. 

#### 예제

In [9]:
sumInt Nil

0

In [10]:
sumInt (Cons 1 Nil)

1

In [11]:
sumInt (Cons 1 (Cons 2 Nil))

3

In [12]:
sumInt (Cons 1 (Cons 2 (Cons 3 Nil)))

6

## 참고사항

재귀적 유형(recursive types)을 귀납적 유형(inductive types) 부르기도 한다.
예를 들어, 많은 수리논리 전공서적과 Coq 증명보조기에서 귀납적 유형이란 표현을 사용하며,
수학 증명을 할 때 귀납적 증명이란 표현을 사용한다.
반면에 재귀란 표현은 함수를 재귀적으로 정의할 때, 즉, 재귀 함수에 대해서만 사용한다. 

하지만 '재귀적'과 '귀납적' 두 표현에 대한 엄밀한 구분은 경우에 따라 불가능하다.
따라서 여기서는 두 개념을 구분하지 않고, 재귀 표현으로 통일해서 사용한다. 